<a href="https://colab.research.google.com/github/Swetha-R-V/AI-Powered-Task-Summarizer-API/blob/main/AI_Powered_Task_Summarizer_APIipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn sqlalchemy psycopg2-binary pydantic python-dotenv pytest httpx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 41.4 MB/s eta 0:00:00


In [ ]:
import os

output_dir = 'app'
output_file = os.path.join(output_dir, 'schemas.py')

os.makedirs(output_dir, exist_ok=True)

content = """from pydantic import BaseModel, Field
from datetime import datetime
from typing import Optional

class TaskCreate(BaseModel):
    title: str = Field(..., min_length=3, example="Daily report")
    description: str = Field(..., min_length=10, example="Prepare daily sales report")

    class Config:
        orm_mode = True


class TaskUpdate(BaseModel):
    title: Optional[str] = Field(None, min_length=3)
    description: Optional[str] = Field(None, min_length=10)

    class Config:
        orm_mode = True


class TaskResponse(BaseModel):
    id: int
    title: str
    description: str
    summary: str
    created_at: datetime

    class Config:
        orm_mode = True
"""

with open(output_file, 'w') as f:
    f.write(content)

print(f"File '{output_file}' created successfully.")


File 'app/schemas.py' created successfully.


In [ ]:
%%writefile app/schemas.py
from pydantic import BaseModel, Field
from datetime import datetime
from typing import Optional

class TaskCreate(BaseModel):
    title: str = Field(..., min_length=3, example="Daily report")
    description: str = Field(..., min_length=10, example="Prepare daily sales report")

    class Config:
        orm_mode = True


class TaskUpdate(BaseModel):
    title: Optional[str] = Field(None, min_length=3)
    description: Optional[str] = Field(None, min_length=10)

    class Config:
        orm_mode = True


class TaskResponse(BaseModel):
    id: int
    title: str
    description: str
    summary: str
    created_at: datetime

    class Config:
        orm_mode = True


Overwriting app/schemas.py


In [ ]:
%%writefile app/external.py
import httpx
from fastapi import HTTPException


def generate_summary(text: str) -> str:
    """
    Calls an external API and returns a short summary text.
    """

    try:
        response = httpx.get(
            "https://api.quotable.io/random",
            timeout=5.0
        )
        response.raise_for_status()

        data = response.json()
        return data.get("content", "Summary not available")

    except httpx.RequestError:
        raise HTTPException(
            status_code=503,
            detail="External API service unavailable"
        )

    except httpx.HTTPStatusError:
        raise HTTPException(
            status_code=502,
            detail="Error from external API"
        )


Writing app/external.py


In [ ]:
%%writefile app/crud.py
from sqlalchemy.orm import Session
from .models import Task
from .schemas import TaskCreate, TaskUpdate
from .external import generate_summary


# CREATE
def create_task(db: Session, task: TaskCreate):
    summary = generate_summary(task.description)

    new_task = Task(
        title=task.title,
        description=task.description,
        summary=summary
    )

    db.add(new_task)
    db.commit()
    db.refresh(new_task)
    return new_task


# READ
def get_task(db: Session, task_id: int):
    return db.query(Task).filter(Task.id == task_id).first()


# UPDATE
def update_task(db: Session, task_id: int, task: TaskUpdate):
    db_task = get_task(db, task_id)

    if db_task is None:
        return None

    if task.title:
        db_task.title = task.title

    if task.description:
        db_task.description = task.description
        db_task.summary = generate_summary(task.description)

    db.commit()
    db.refresh(db_task)
    return db_task


# DELETE
def delete_task(db: Session, task_id: int):
    db_task = get_task(db, task_id)

    if db_task is None:
        return None

    db.delete(db_task)
    db.commit()
    return True


Writing app/crud.py


In [20]:
%%writefile app/main.py
from fastapi import FastAPI, Depends, HTTPException
from sqlalchemy.orm import Session
from fastapi.responses import JSONResponse
from fastapi import Request

from .database import SessionLocal
from . import crud, schemas

app = FastAPI(title="AI Task Summarizer API")


# Dependency to get DB session
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


# 1⏸ CREATE TASK
@app.post("/tasks", response_model=schemas.TaskResponse, status_code=201)
def create_task(task: schemas.TaskCreate, db: Session = Depends(get_db)):
    return crud.create_task(db, task)


# 2⏸ GET TASK
@app.get("/tasks/{task_id}", response_model=schemas.TaskResponse)
def get_task(task_id: int, db: Session = Depends(get_db)):
    task = crud.get_task(db, task_id)
    if task is None:
        raise HTTPException(status_code=404, detail="Task not found")
    return task


# 3⏸ UPDATE TASK
@app.put("/tasks/{task_id}", response_model=schemas.TaskResponse)
def update_task(task_id: int, task: schemas.TaskUpdate, db: Session = Depends(get_db)):
    updated_task = crud.update_task(db, task_id, task)
    if updated_task is None:
        raise HTTPException(status_code=404, detail="Task not found")
    return updated_task


# 4⏸ DELETE TASK
@app.delete("/tasks/{task_id}", status_code=204)
def delete_task(task_id: int, db: Session = Depends(get_db)):
    result = crud.delete_task(db, task_id)
    if result is None:
        raise HTTPException(status_code=404, detail="Task not found")
    return


# Global exception handler
@app.exception_handler(Exception)
async def global_exception_handler(request: Request, exc: Exception):
    return JSONResponse(
        status_code=500,
        content={"detail": "Internal Server Error"}
    )

Overwriting app/main.py


In [28]:
%%writefile app/database.py
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

# Configure the database URL
# For simplicity, we'll use a SQLite database file named 'sql_app.db'
# in the same directory as the script. For production, you'd typically
# use an environment variable for a PostgreSQL or MySQL connection string.
SQLALCHEMY_DATABASE_URL = "sqlite:///./sql_app.db"

# Create the SQLAlchemy engine.
# The connect_args is needed for SQLite to allow multiple threads to interact
# with the database if using a single connection. This is common for FastAPI.
engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)

# Create a SessionLocal class.
# Each instance of SessionLocal will be a database session.
# The autocommit=False ensures that we have to commit changes explicitly.
# The autoflush=False means that query operations won't automatically flush pending changes to the database.
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Create a Base class for declarative models.
# This is where our SQLAlchemy models will inherit from.
Base = declarative_base()


Writing app/database.py


In [29]:
import os
from app.database import engine, Base
from app import models

# Create database tables
Base.metadata.create_all(bind=engine)

print("Database tables created successfully.")

Database tables created successfully.


In [31]:
!pip install pytest httpx


In [34]:
%%writefile tests/test_tasks.py
from fastapi.testclient import TestClient
from unittest.mock import patch

from app.main import app

client = TestClient(app)


def test_create_task():
    with patch("app.external.generate_summary") as mock_summary:
        mock_summary.return_value = "Test summary"

        response = client.post(
            "/tasks",
            json={
                "title": "Test Task",
                "description": "This is a test task description"
            }
        )

        assert response.status_code == 201
        data = response.json()
        assert data["title"] == "Test Task"
        assert data["summary"] == "Test summary"


def test_get_task_not_found():
    response = client.get("/tasks/9999")
    assert response.status_code == 404


Writing tests/test_tasks.py
